In [2]:
import pandas as pd
import json
import numpy as np

In [3]:
test_file = "faa_eval_faa.json"
ner_model = "PL-Marker/sciner_models/sciner-scibert-faa"
re_model = "PL-Marker/scire_models/scire-scibert-faa"

In [4]:
with open(f"PL-Marker/scierc/{test_file}") as f:
    test_str = f.read()

test = []
for test_doc_str in test_str.split('\n'):
    try:
        test.append(json.loads(test_doc_str))
    except:
        print(f"could not parse: {test_doc_str}")

could not parse: 


In [6]:
len(test[1]['sentences'])

3

In [7]:
idx2word = {}
for idoc in range(len(test)):
    doc = test[idoc]

    idx2word[idoc] = {}
    curr_idx = 0
    for sentence in test[idoc]['sentences']:
        for word in sentence:
            if word not in idx2word[idoc]:
                idx2word[idoc][curr_idx] = word
                curr_idx = curr_idx + 1

In [8]:
with open(f"{ner_model}/ent_pred_test.json") as f:
    ent_pred_str = f.read()

ent_pred = []
for ent_pred_doc_str in ent_pred_str.split('\n'):
    try:
        ent_pred.append(json.loads(ent_pred_doc_str))
    except:
        print(f"could not parse: {ent_pred_doc_str}")

could not parse: 


In [9]:
ent_pred[0].keys()

dict_keys(['sentences', 'ner', 'relations', 'doc_key', 'predicted_ner'])

In [12]:
len(ent_pred[1]['sentences'])

3

In [13]:
def get_span(idoc, sentence, idx0, idx1):
    start_span = sentence.index(idx2word[idoc][idx0])
    end_span = sentence.index(idx2word[idoc][idx1], start_span)
    while True:
        try:
            start_span = sentence.index(idx2word[idoc][idx0], start_span+1, end_span)
        except:
            break
    return sentence[start_span:end_span+1]

In [14]:
for idoc in range(len(ent_pred)):
    print(f"*** Document #{idoc} ****")
    for isentence in range(len(ent_pred[idoc]["sentences"])):
        print(' '.join(ent_pred[idoc]['sentences'][isentence]))
        print("Correct:")
        for entity in ent_pred[idoc]['ner'][isentence]:
            print(entity)
            span = ' '.join(get_span(idoc, ent_pred[idoc]['sentences'][isentence], entity[0], entity[1]))
            print(f"{span} : {entity[2]}")
        print("\nPredicted:")
        for entity in ent_pred[idoc]['predicted_ner'][isentence]:
            print(entity)
            span = ' '.join(get_span(idoc, ent_pred[idoc]['sentences'][isentence], entity[0], entity[1]))
            print(f"{span} : {entity[2]}")
        print("\n----------------\n")

*** Document #0 ****
tailwheel cocked right prior to tkof .
Correct:

Predicted:
[0, 0, 'OtherScientificTerm']
tailwheel : OtherScientificTerm

----------------

*** Document #1 ****
tow plane became airborne then settled .
Correct:

Predicted:

----------------

student thought tow in trouble & released .
Correct:

Predicted:

----------------

hit tree .
Correct:

Predicted:

----------------

*** Document #2 ****
2nd ils apch , acft 's g/s inop .
Correct:

Predicted:

----------------

lom tuned to wrong freq .
Correct:

Predicted:

----------------

*** Document #3 ****
plt noted soft r brake pedal drg taxi to tkof .
Correct:

Predicted:

----------------

flt rtnd springfield due soft brake strong winds bos .
Correct:

Predicted:
[15, 16, 'OtherScientificTerm']
soft brake : OtherScientificTerm

----------------

*** Document #4 ****
taxi off hard sfc due tfc r main gr broke through roof of a washed out underground tunnel .
Correct:

Predicted:

----------------

*** Document #5 **

In [15]:
with open(f"{re_model}/pred_results.json") as f:
    pred_results_str = f.readlines()[0]
    pred_results = json.loads(pred_results_str)

In [16]:
pred_results.keys()

dict_keys(['0', '3', '7', '9', '10', '11', '13', '17', '19', '25', '26', '27', '28', '30', '31', '34', '35', '37', '38', '40', '45', '47', '49', '53', '54', '56', '58', '61', '63', '64', '68', '69', '70', '71', '72', '74', '77', '78', '80', '81', '82', '83', '85', '89', '90', '94', '101', '105', '106', '110', '111', '119', '120', '124', '126', '129', '130', '133', '136', '141', '142', '143', '145', '147', '148', '149', '150', '153', '154', '157', '159', '160', '163', '164', '165', '167', '169', '177', '179', '180', '181', '184', '187', '191', '194', '199', '201', '202', '203', '207', '209', '210', '211', '213', '214', '216', '217', '221', '224', '225', '227', '231', '233', '235', '236', '240', '241', '243', '245', '246', '247', '248', '250', '253', '258', '259', '261', '262', '263', '266', '267', '270', '271', '273', '278', '280', '281', '283', '289', '290', '294', '295', '297', '298', '299', '300', '301', '304', '307', '315', '321', '325', '326', '331', '332', '333', '336', '338', '33

In [17]:
for doc_str in pred_results.keys():
    idoc = int(doc_str)
    print(f"*** Document #{idoc} ****")
    for isentence in range(len(pred_results[f'{idoc}'])):
        sentence_num = pred_results[f'{idoc}'][isentence][0]
        triples = pred_results[f'{idoc}'][isentence][1]
        sentence = ent_pred[idoc]['sentences'][sentence_num]
        print(' '.join(sentence))
        if len(triples) > 0:
            for triple in triples:
                print(f"\t{' '.join(get_span(idoc, sentence, triple[0][0], triple[0][1]))} --> {triple[2]} --> {' '.join(get_span(idoc, sentence, triple[1][0], triple[1][1]))}")
        else:
            print("\tNo triples found")
        print("\n--------------\n")

*** Document #0 ****
tailwheel cocked right prior to tkof .
	No triples found

--------------

*** Document #3 ****
flt rtnd springfield due soft brake strong winds bos .
	No triples found

--------------

*** Document #7 ****
mtns obscured .
	No triples found

--------------

*** Document #9 ****
turbocharger waste gate did not actuate .
	No triples found

--------------

*** Document #10 ****
gear-up ldg in mesquite brush .
	No triples found

--------------

*** Document #11 ****
plt continued t/o ifr after wndshd & windows fogged up during vfr departure .
	No triples found

--------------

*** Document #13 ****
elect syst went dead , generator circuit breaker switch not engaged .
	No triples found

--------------

*** Document #17 ****
r eng , l mag excessive drop .
	No triples found

--------------

*** Document #19 ****
r rudder cable caught under battery box support structure .
	No triples found

--------------

*** Document #25 ****
acft n42807 & n7930c substantially dmgd .
	No 

In [23]:
results_dict = {"doc_idx":[], "sent_idx":[], "original_sentence":[], "input":[], "head":[],"relation":[], "tail":[]}
og_data = pd.read_csv("../../data/FAA_data/Maintenance_Text_data_nona.csv")

for doc_str in pred_results.keys():
    idoc = int(doc_str)
    for isentence in range(len(pred_results[f'{idoc}'])):
        sentence_num = pred_results[f'{idoc}'][isentence][0]
        triples = pred_results[f'{idoc}'][isentence][1]
        sentence = ent_pred[idoc]['sentences'][sentence_num]
        og_sentence = og_data["c119"].iat[idoc]

        for triple in triples:
            results_dict["doc_idx"].append(idoc)
            results_dict["sent_idx"].append(isentence)
            results_dict["original_sentence"].append(og_sentence)
            results_dict["input"].append(' '.join(sentence))
            results_dict["head"].append(' '.join(get_span(idoc, sentence, triple[0][0], triple[0][1])))
            results_dict["relation"].append(triple[2])
            results_dict["tail"].append(' '.join(get_span(idoc, sentence, triple[1][0], triple[1][1])))

In [26]:
for i_triple in range(len(results_dict["doc_idx"])):
    print(f"{results_dict['doc_idx'][i_triple]}, {results_dict['original_sentence'][i_triple]}\t---from---\t{og_data["c119"].iat[results_dict['doc_idx'][i_triple]]}")

26, PLT DID NOT USE EMERG LNDNG GR EXTNSN AFTER ELECTRICAL PWR FAILURE.                                                	---from---	PLT DID NOT USE EMERG LNDNG GR EXTNSN AFTER ELECTRICAL PWR FAILURE.                                                
34, FORCED LANDING AFTER POWER LOSS. WATER FOUND IN FUEL.                                                              	---from---	FORCED LANDING AFTER POWER LOSS. WATER FOUND IN FUEL.                                                              
37, ENGINE QUIT, LANDING MADE ON FROZEN LAKE. ICE RESTRICTION IN MAIN FUEL STRAINER.                                   	---from---	ENGINE QUIT, LANDING MADE ON FROZEN LAKE. ICE RESTRICTION IN MAIN FUEL STRAINER.                                   
58, ENGINE STOPPED SHORTLY AFTER TAKEOFF FROM REFUELING. SYSTEM, GASCOLATOR AND CARBURETOR CONTAINED WATER.            	---from---	ENGINE STOPPED SHORTLY AFTER TAKEOFF FROM REFUELING. SYSTEM, GASCOLATOR AND CARBURETOR CONTAINED WATER.            
58, ENGINE S

In [27]:
pd.DataFrame(results_dict).to_csv(f"../../data/results/pl-marker/pl-marker_results.csv")